# Building a Collaborative Filtering news recommender
In this notebook we preprocess and train a news recommender system based on the MIND dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch

## Manual pre-processing of data
There exist a library to preprocess this data, but I find it quite hard to use it. So instead, after a quick look on the data I found it would not be too hard to do the work myself.
For the first read, it is not necessary to understand what is being done here, and I will re-introduce the dataset in the modeling section.

### behaviors.tsv

#### From documentation:  
The behaviors.tsv file contains the impression logs and users' news click histories. It has 5 columns divided by the tab symbol:
- Impression ID. The ID of an impression.  
- User ID. The anonymous ID of a user.  
- Time. The impression time with format "MM/DD/YYYY HH:MM:SS AM/PM".  
- History. The news click history (ID list of clicked news) of this user before this impression. The clicked news articles are ordered by time.  
- Impressions. List of news displayed in this impression and user's click behaviors on them (1 for click and 0 for non-click). The orders of news in a impressions have been shuffled.  

In [ ]:
raw_behaviour = pd.read_csv(
    "/kaggle/input/mind-news-dataset/MINDsmall_train/behaviors.tsv", 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])

print(f"The dataset consist of {len(raw_behaviour)} number of interactions.")
raw_behaviour.head()

In [ ]:
## Indexize users
unique_userIds = raw_behaviour['userId'].unique()
# Allocate a unique index for each user, but let the zeroth index be a UNK index:
ind2user = {idx +1: itemid for idx, itemid in enumerate(unique_userIds)}
user2ind = {itemid : idx for idx, itemid in ind2user.items()}
print(f"We have {len(user2ind)} unique users in the dataset")

# Create a new column with userIdx:
raw_behaviour['userIdx'] = raw_behaviour['userId'].map(lambda x: user2ind.get(x,0))

In [ ]:
raw_behaviour.head()

## Load article data
We also need to get the content information of each article. We will use the news.tsv file to index the items.

In [ ]:
news = pd.read_csv(
    "/kaggle/input/mind-news-dataset/MINDsmall_train/news.tsv", 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
news.head(2)
# Build index of items
ind2item = {idx +1: itemid for idx, itemid in enumerate(news['itemId'].values)}
item2ind = {itemid : idx for idx, itemid in ind2item.items()}

news.head()

Now we need to process the click history and impressions. We need to both indexize the strings, but also to decode impressions into clicks and non-clicks.



In [ ]:
# Indexize click history field
def process_click_history(s):
    list_of_strings = str(s).split(" ")
    return [item2ind.get(l, 0) for l in list_of_strings]
        
raw_behaviour['click_history_idx'] = raw_behaviour.click_history.map(lambda s:  process_click_history(s))
raw_behaviour.head()

In [ ]:
# collect one click and one no-click from impressions:
def process_impression(s):
    list_of_strings = s.split(" ")
    itemid_rel_tuple = [l.split("-") for l in list_of_strings]
    noclicks = []
    for entry in itemid_rel_tuple:
        if entry[1] =='0':
            noclicks.append(entry[0])
        if entry[1] =='1':
            click = entry[0]
    return noclicks, click

raw_behaviour['noclicks'], raw_behaviour['click'] = zip(*raw_behaviour['impressions'].map(process_impression))
# We can then indexize these two new columns:
raw_behaviour['noclicks'] = raw_behaviour['noclicks'].map(lambda list_of_strings: [item2ind.get(l, 0) for l in list_of_strings])
raw_behaviour['click'] = raw_behaviour['click'].map(lambda x: item2ind.get(x,0))

In [ ]:
raw_behaviour.head()

In [ ]:
# convert timestamp value to hours since epoch
raw_behaviour['epochhrs'] = pd.to_datetime(raw_behaviour['timestamp']).values.astype(np.int64)/(1e6)/1000/3600
raw_behaviour['epochhrs'] = raw_behaviour['epochhrs'].round()

## find first publish date
#raw_behaviour[['click','epochhrs']].groupby("click").min("epochhrs").reset_index()

## Output of data preprocessing
In this preprocessing we have processed behaviour data, article data and user data.
Most importantly, we have indexized users and items in the behaviour dataframe, 
as pytorch requires integer indicies instead of strings for user and item IDs.

- Dataframe`behaviour` contains all interactions: epochhrs (a timestamp), clicks, no clicks and historical clicks per user for each interaction
- Dictionary `ind2item`: mapping the item indicies given in behaviour to the real item Id given in the dataset.
- Dictionary `ind2user`: mapping the user indicies given in behaviour to the real user Id given in the dataset.
- Dataframe `news`: content data on items. We will not use this in the first iteration.

The main component is `behaviour`, and for collaborative filtering purposes this is all we need.
However, if we want to utilize content data on the news items some preprocessing on the `news` dataframe must be used.

In [ ]:
## Select the columns that we now want to use for further analysis
behaviour = raw_behaviour[['epochhrs','userIdx','click_history_idx','noclicks','click']]
behaviour.head()

# Modeling
We want to make a matrix factorization model where each user $u$ has a d-dimensional parameter vector $z_u$ and each item $i$ has a parameter vector $v_i$. This implies that we do not need the click_history_idx column when we train the model (why?).

Second, to simplify the computation of things we assume that the user only considers two items in each interaction: The item the user clicked on, and the first item in the noclicks list (what are we missing by this assumption?).

Hence, our dataset consist of a `behaviour` dataframe and a `news` dataframe with content information of the news articles. We will use a language model to interpret our article data, and for simplicity we will only use the title as the text input here.


In [ ]:
behaviour.loc[:,'noclick'] = behaviour['noclicks'].map(lambda x : x[0])
behaviour.head()

In [ ]:
# Let us use the last 10pct of the data as our validation data:
test_time_th = behaviour['epochhrs'].quantile(0.9)
train = behaviour[behaviour['epochhrs']< test_time_th]
valid =  behaviour[behaviour['epochhrs']>= test_time_th]

In [ ]:
class MindDataset(Dataset):
    # A fairly simple torch dataset module that can take a pandas dataframe (as above), 
    # and convert the relevant fields into a dictionary of arrays that can be used in a dataloader
    def __init__(self, df):
        # Create a dictionary of tensors out of the dataframe
        self.data = {
            'userIdx' : torch.tensor(df.userIdx.values),
            'click' : torch.tensor(df.click.values),
            'noclick' : torch.tensor(df.noclick.values)
        }
    def __len__(self):
        return len(self.data['userIdx'])
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.data.items()}

In [ ]:
# Build datasets and dataloaders of train and validation dataframes:
bs = 1024
ds_train = MindDataset(df=train)
train_loader = DataLoader(ds_train, batch_size=bs, shuffle=True)
ds_valid = MindDataset(df=valid)
valid_loader = DataLoader(ds_valid, batch_size=bs, shuffle=False)

batch = next(iter(train_loader))

## Model

#### Framework
We will use pytorch-lightning to define and train our model. It is a high-level framework (similar to fastAI) but with a slightly different way of defining things. It is my personal go-to framework and is very flexible. For more information, see https://pytorch-lightning.readthedocs.io/.

#### The model
We assume that each interaction goes as follow: the user is presented with two items: the click and no-click item. After the user reviewed both items, she will choose the most relevant one. This can be modeled as a categorical distirbution with two options (yes, you could do binomial). There is a loss function in pytorch for this already, called the `F.cross_entropy` that we will use.

In [ ]:
# Build a matrix factorization model
class NewsMF(pl.LightningModule):
    def __init__(self, num_users, num_items, dim = 10):
        super().__init__()
        self.dim=dim
        self.useremb = nn.Embedding(num_embeddings=num_users, embedding_dim=dim)
        self.itememb = nn.Embedding(num_embeddings=num_items, embedding_dim=dim)

    def step(self, batch, batch_idx, phase="train"):
        batch_size = batch['userIdx'].size(0)
        score_click = self.forward(batch["userIdx"], batch["click"])
        score_noclick = self.forward(batch["userIdx"], batch["noclick"])         
        scores_all = torch.concat((score_click, score_noclick), dim=1)
        loss = F.cross_entropy(input=scores_all, target=torch.zeros(batch_size, device=scores_all.device).long())
        return loss
    
    def forward(self, users, items):
        uservec =  self.useremb(users)
        itemvec = self.itememb(items)
        score = (uservec*itemvec).sum(-1).unsqueeze(-1)
        return score
               
        
    def predict_single_user(self, user_idx):
        items = torch.arange(0, len(ind2item))
        user = torch.zeros_like(items) + user_idx
        scores = self.forward(user, items)
        recommendations = [item.item() for item in torch.topk(scores, 500, dim=0)[1]]
        return recommendations
    
    def predict(self, users):
        recommendations = []
        for user in users:
            recommendation = self.predict_single_user(user)
            recommendations.append(recommendation) 
        return recommendations        
        
    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, "train")
    
    def validation_step(self, batch, batch_idx):
        # for now, just do the same computation as during training
        return self.step(batch, batch_idx, "val")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
mf_model = NewsMF(num_users=len(ind2user)+1, num_items = len(ind2item)+1, dim=15)

In [ ]:
trainer = pl.Trainer(max_epochs=100, accelerator="gpu")
trainer.fit(model=mf_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

## Performance

Here we get a batch of test data and calculate some performance metrics. We only have 1 true value (the item that the user actually clicked), so let's define our own version of accuracy@k as follows:

- If the item that the user clicked is in the top 10 predictions, return 1,
- Otherwise, return 0.

The mean of all predictions are a sort of accuracy@k metric. The difficulty here is of course that we only have 1 true value, otherwise we would use something like precision@k or recall@k, but those don't apply here

In [ ]:
valid_batch = next(iter(valid_loader))
predictions = mf_model.predict(valid_batch["userIdx"])
true_values = [item.item() for item in valid_batch["click"]]

In [ ]:
from typing import List

def accuracy_at_k(predictions: List[List], true_values: List):
    hits = 0
    for preds, true in zip(predictions, true_values):
        if true in preds:
            hits += 1
    return hits / len(true_values)

accuracy_at_k(predictions, true_values)

## Playground

The cells below are a playground to inspect single recommendations

In [ ]:
user_idx = 5
items = torch.arange(0, len(ind2item))
user = torch.zeros_like(items) + user_idx
recommendations = mf_model.predict_single_user(user)

In [ ]:
# read news article
article_id = behaviour[behaviour.userIdx == user_idx]["click"].values[0]
news[news.itemId == ind2item[article_id]]

In [ ]:
news[news.itemId.isin([(ind2item[item + 1])  for item in recommendations])]